# Code for Lecture-1 of Short Course of Temporal Point Processes

In [4]:
import numpy as np
import torch

In [18]:
# fix random seed 
np.random.seed(12345)
torch.random.manual_seed(12345)

## Univariate Point Process with Constant Intensity

### Sampling a sequence of events

In [12]:
# function of sampling next event given constant intensity 
# method : inversion sampling 
def draw_dt_invsam(intensity): 
    u = np.random.uniform(low=0.0, high=1.0)
    return -np.log(1-u)/intensity

In [49]:
# draw a sequence over [0, T]
# from a point process with constant intensity
T = 1000.0
intensity = 0.5
t = 0
seq = []

while True:
    dt = draw_dt_invsam(intensity)
    t += dt
    if t <= T: 
        seq += [t]
    else: 
        break

# check the data 
print(f"over time interval [0, {T}]")
print(f"# of events : {len(seq)}")
print(f"time of 0-th event : {seq[0]:.4f}")
print(f"time of {len(seq)-1}-th event : {seq[-1]:.4f}")

over time interval [0, 1000.0]
# of events : 496
time of 0-th event : 4.4187
time of 495-th event : 995.1007


### Estimate intensity by maximum log-likelihood estimation (MLE)

When intensity is a constant, the MLE is ${I}/(T - t_0)$ where $I$ is the number events and $[t_0, T]$ is the observation interval

In [50]:
# MLE : # of events / (T - t_0)

intensity_mle = len(seq) / (T - 0.0)

# check the estimate
print(f"true intenstiy is : {intensity}")
print(f"estimate by MLE is : {intensity_mle}")

true intenstiy is : 0.5
estimate by MLE is : 0.496


### Predict next event time by minimum Bayes risk (MBR)

MBR prediction for next event time is $\int_{s}^{\infty} t p(t) dt$ where $s$ is the starting time and $p(t) = \lambda \exp(-\lambda (t-s))$ is the density function of next event time $t \geq s$. 
When $\lambda$ is constant, it is as simple as $(\lambda s + 1) / \lambda = s + 1/\lambda$.
So it is the starting time $s$ plus the expected $dt$ with given intensity $\lambda$.  

In [51]:
se_mle, se_true = 0.0, 0.0
for i, s in enumerate([0.0] + seq[:-1]): 
    t_pred_mle = s + 1.0 / intensity_mle
    t_pred_true = s + 1.0 / intensity
    t = seq[i]
    se_mle += (t_pred_mle - t) ** 2
    se_true += (t_pred_true - t) ** 2

rmse_mle = np.sqrt(se_mle / len(seq))
rmse_true = np.sqrt(se_true / len(seq))

print(f"check time prediction accuracy")
print(f"RMSE using estimated intensity : {rmse_mle:.4f}")
print(f"RMSE using true intensity : {rmse_true:.4f}")

check time prediction accuracy
RMSE using estimated intensity : 2.3414
RMSE using true intensity : 2.3414


## Multivariate Point Process with Constant Intensities

### Sampling a sequence of events¶

In [52]:
# draw a sequence over [0, T]
# from a point process with constant intensity
T = 1000.0
intens = [0.75, 0.25]
intensity = sum(intens)
probs = np.array(intens) / intensity
t = 0
seq = []

while True:
    # draw dt using inversion sampling
    dt = draw_dt_invsam(intensity)
    t += dt
    if t <= T: 
        # draw type from categorical dist
        k = np.random.choice(2, p=probs)
        seq += [(t, k)]
    else: 
        break

# check the data 
c0 = 0
for x in seq: 
    if x[1] == 0: 
        c0 += 1
c1 = len(seq) - c0
print(f"over time interval [0, {T}]")
print(f"# of events : {len(seq)}")
print(f"time of 0-th event : {seq[0][0]:.4f}")
print(f"time of {len(seq)-1}-th event : {seq[-1][0]:.4f}")
print(f"# of type-0 : {c0}")
print(f"# of type-1 : {c1}")

over time interval [0, 1000.0]
# of events : 1009
time of 0-th event : 0.2168
time of 1008-th event : 999.0822
# of type-0 : 754
# of type-1 : 255


### Estimate intensities by MLE

When intensities are constants, the MLE is $\lambda_0 = {I}_0/(T - t_0)$ and $\lambda_1 = I_1 / (T - t_0)$ where $I_k$ is the number events of $k$-th type and $[t_0, T]$ is the observation interval. 

In [57]:
# MLE : # of events / (T - t_0)

inten0_mle = c0 / (T - 0.0)
inten1_mle = c1 / (T - 0.0)
intensity_mle = inten0_mle + inten1_mle

# check the estimate
print(f"true intensties are : {intens}")
print(f"estimates by MLE is : [{inten0_mle:.2f}, {inten1_mle:.2f}]")

true intensties are : [0.75, 0.25]
estimates by MLE is : [0.75, 0.26]


### Predict next event time and type by minimum Bayes risk (MBR)

MBR prediction for next event time is $\int_{s}^{\infty} t p(t) dt$ where $s$ is the starting time, $p(t) = \lambda \exp(-\lambda (t-s))$ is the density function of next event time $t \geq s$, and $\lambda = \lambda_0 + \lambda_1$ is the total intensity. 
When $\lambda_0$ and $\lambda_1$ are constants, it is as simple as $(\lambda s + 1) / \lambda = s + 1/\lambda$, the same as in the univariate case because time prediction is only dependent on the total intensity, but not individual intensities. 

MBR prediction for type given time is $\arg\!\max_k \lambda_k$. 

In [58]:
se_mle, se_true = 0.0, 0.0
nerr_mle, nerr_true = 0, 0 
for i, s in enumerate([(0.0, None)] + seq[:-1]): 
    # time
    t_pred_mle = s[0] + 1.0 / intensity_mle
    t_pred_true = s[0] + 1.0 / intensity
    t = seq[i][0]
    se_mle += (t_pred_mle - t) ** 2
    se_true += (t_pred_true - t) ** 2
    # type 
    k_mle = np.argmax([inten0_mle, inten1_mle])
    k_true = np.argmax(intens)
    k = seq[i][1]
    if k_mle != k: 
        nerr_mle += 1
    if k_true != k: 
        nerr_true += 1

rmse_mle = np.sqrt(se_mle / len(seq))
rmse_true = np.sqrt(se_true / len(seq))

print(f"check time prediction accuracy")
print(f"RMSE using estimated intensity : {rmse_mle:.4f}")
print(f"RMSE using true intensity : {rmse_true:.4f}")

print(f"\ncheck type prediction error rate")
print(f"Error Rate using estimated intensities : {100.0*nerr_mle/len(seq):.2f}%")
print(f"Error Rate using true intensities : {100.0*nerr_true/len(seq):.2f}%")


check time prediction accuracy
RMSE using estimated intensity : 1.0192
RMSE using true intensity : 1.0192

check type prediction error rate
Error Rate using estimated intensities : 25.27%
Error Rate using true intensities : 25.27%


Note: this model will always predict $k=0$ because $\lambda_0 > \lambda_1$ holds for all the infinitely many time points. 
This model is of course bad because it doesn't consider any context. 
We will walk through better models in the next lecture.